# Setup 

## import libraries

In [ ]:
import numpy as np
import os 
import pandas as pd
import json
from langchain_community.llms import Ollama
from langchain import PromptTemplate
import re
import ast

## import data

In [ ]:
athenadata = open("Downloads/Athena_feedback.txt", encoding='windows-1252').read().splitlines()

In [ ]:
UserID = []
Message = []

for i in range(0,len(athenadata)):
    if i % 2 == 0:
        UserID.append(athenadata[i])
    elif i % 2 == 1:
        Message.append(athenadata[i])
    else:
        continue

In [ ]:
AUF_Df = pd.DataFrame({
    'UserID':UserID,
    'Message':Message
})

In [ ]:
AUF_Df.head()

## Select LLM

In [ ]:
# llm = Ollama(base_url='http://localhost:11434',
#              model="llama2")
# mistral_bot = Ollama(base_url='http://localhost:11434',
#              model="mistral")
ActiveLLM = Ollama(base_url='http://localhost:11434',
             model="llama3.1")


## Initialize prompt

In [ ]:
UX_metathemeUXR_prompt = """
Role: You are in the role of a UX researcher evaluating user feedback to find what users like about our new software system. Those likes and dislikes will be eventually turned into design decisions.
Our system is an AI system that helps user navigate a database of petroleum engineering documentation.

Instructions:
You will be given a piece of userfeedback, and notes on what has been in the user feedback data so far.  Analyze the piece of user feedback in the context of the notes. This feedback may be long and cover 1 -4 main topics. Write a new summary of what has been seen so far based on the notes and the current piece of feedback. The new summary should contain the themes that have been seen so far and how much evidence there is for theme, and any new themes from the current piece of feedback.
For example if the notes say that “users find the search function frustrating” and the piece of feedback you are analyzing says “search does not work properly” then the new notes you write down should capture how often you are finding evidence for the theme of frustration with the search function.  
Try to keep the themes limited to 1-7 important themes. the themes should be concise and not redundant. Refactor the themes as needed as you gain more informaiton in the notes section.

Example:
	Example Notes: So far we have analyzed 6 pieces of user feedback. And we have found that users are finding the writing function to be poor (five times), and the drawing function to be excellent (two times)

	Example user feedback:
		The drawing function is excellent. The video function is excellent.
	
	Example Output:
		So far we have analyzed 7 pieces of user feedback. And we have found that users are finding the writing function to be poor (5 times), and the drawing function to be excellent (3 times). And the video function is also excellent (1 time).

Notes: {notes}

User feedback: {userfeedback}

Output: """


In [ ]:
prompt_templateRolling = PromptTemplate(
    input_variables=["notes","userfeedback"],
    template=UX_metathemeUXR_prompt
)

In [ ]:
def use_rollingtopicanalyzerLLM(input1, input2, template):

    # print(template.format(userfeedback=input))
    
    output = ActiveLLM(
    template.format(
        notes=input1,
        userfeedback=input2
        )
    )
    return output

# use actually process data

## get initial topics
notice that we processed just one message first because this is rather slow, 
so I recommend when you are adapting your prompt to try it on one message at a time first 
before running all of them as it take 30 minutes to an hour to run for everyone

In [ ]:
AUF_Dft1 = AUF_Df.iloc[0:2,]

In [ ]:
AUF_Dft1['Message'][0]

In [ ]:
Notes = "no notes yet"
for i in range(0, len(AUF_Dft1)):
    Notes = use_rollingtopicanalyzerLLM(input1=Notes,input2=AUF_Dft1['Message'][i], template=prompt_templateRolling)

In [ ]:
Notes

In [ ]:
Notes = "no notes yet"
for i in range(0, len(AUF_Df)):
    Notes = use_rollingtopicanalyzerLLM(input1=Notes,input2=AUF_Df['Message'][i], template=prompt_templateRolling)

In [ ]:
Notes

In [ ]:
AUF_Dft1['Topics1'] = AUF_Dft1['Message'].apply(use_topicanalyzerLLM, template=prompt_templateT)

In [ ]:
AUF_Dft1['Topics1'][0]

In [ ]:
AUF_Dft1['Topics1'] = AUF_Dft1['Message'].apply(use_topicanalyzerLLM, template=prompt_templateT)
AUF_Dft1['Topics2'] = AUF_Dft1['Message'].apply(use_topicanalyzerLLM, template=prompt_templateT)
AUF_Dft1['Topics3'] = AUF_Dft1['Message'].apply(use_topicanalyzerLLM, template=prompt_templateT)
AUF_Dft1['Topics4'] = AUF_Dft1['Message'].apply(use_topicanalyzerLLM, template=prompt_templateT)
AUF_Dft1['Topics5'] = AUF_Dft1['Message'].apply(use_topicanalyzerLLM, template=prompt_templateT)

In [ ]:
AUF_Dft1['combinedTopics'] = 'Researcher 1 :' + AUF_Dft1['Topics1'].astype(str) + 'Researcher 2:' + AUF_Dft1['Topics2'].astype(str) + 'Researcher 3:' + AUF_Dft1['Topics3'].astype(str) + 'Researcher 4:' + AUF_Dft1['Topics4'].astype(str) + 'Researcher 5:' + AUF_Dft1['Topics5'].astype(str)

In [ ]:
AUF_Dft1['combinedTopics'][0]

In [ ]:
AUF_Dft1['finalTopics'] = AUF_Dft1['combinedTopics'].apply(use_scLLM, template=prompt_templateSC)

In [ ]:
# AUF_Dft1['combinedTopics'] = 'Researcher 1 :' + AUF_Dft1['Topics1'].astype(str) + 'Researcher 2:' + AUF_Dft1['Topics2'].astype(str) + 'Researcher 3:' + AUF_Dft1['Topics3'].astype(str) + 'Researcher 4:' + AUF_Dft1['Topics4'].astype(str) + 'Researcher 5:' + AUF_Dft1['Topics5'].astype(str)

In [ ]:
topics_list_test = AUF_Dft1['finalTopics'].tolist()

In [ ]:
Majortopics_test = useLLM_topicSynthesizer( topics_list_test ,template=prompt_templateMTY)

In [ ]:
Majortopics_test

## do full df

In [ ]:
AUF_Df['Topics1'] = AUF_Df['Message'].apply(use_topicanalyzerLLM, template=prompt_templateT)
AUF_Df['Topics2'] = AUF_Df['Message'].apply(use_topicanalyzerLLM, template=prompt_templateT)
AUF_Df['Topics3'] = AUF_Df['Message'].apply(use_topicanalyzerLLM, template=prompt_templateT)
AUF_Df['Topics4'] = AUF_Df['Message'].apply(use_topicanalyzerLLM, template=prompt_templateT)
AUF_Df['Topics5'] = AUF_Df['Message'].apply(use_topicanalyzerLLM, template=prompt_templateT)

In [ ]:
AUF_Df['combinedTopics'] = 'Researcher 1 :' + AUF_Df['Topics1'].astype(str) + 'Researcher 2:' + AUF_Df['Topics2'].astype(str) + 'Researcher 3:' + AUF_Df['Topics3'].astype(str) + 'Researcher 4:' + AUF_Df['Topics4'].astype(str) + 'Researcher 5:' + AUF_Df['Topics5'].astype(str)

In [ ]:
AUF_Df['finalTopics'] = AUF_Df['combinedTopics'].apply(use_scLLM, template=prompt_templateSC)

In [ ]:
topics_list = []
for i  in range(0, len(AUF_Df), 10):
    topics_list.append(" ".join(AUF_Df['finalTopics'][i:i+10]))

In [ ]:
topics_condensed = []
for x in topics_list:
    topics_condensed.append(useLLM_topicSynthesizer( x ,template=prompt_templateMTY))

In [ ]:
Majortopics = useLLM_topicSynthesizer( " ".join(topics_condensed) ,template=prompt_templateMTY)

In [ ]:
topics_list = AUF_Df['finalTopics'].tolist()

In [ ]:
# topics_list

In [ ]:
len(topics_list)

In [ ]:
len(" ".join(topics_list))/4

In [ ]:
Majortopics = useLLM_topicSynthesizer( topics_list ,template=prompt_templateMTY)

In [ ]:
Majortopics

In [ ]:
AUF_Df['finalTopics2'] = AUF_Df['combinedTopics'].apply(use_scLLM, template=prompt_templateSC)

In [ ]:
topics_list2 = []
for i  in range(0, len(AUF_Df), 10):
    topics_list2.append(" ".join(AUF_Df['finalTopics2'][i:i+10]))

In [ ]:
topics_condensed2 = []
for x in topics_list2:
    topics_condensed2.append(use_summLLM( x ,template=prompt_templateSumm))

In [ ]:
Majortopics2 = useLLM_topicSynthesizer( " ".join(topics_condensed2) ,template=prompt_templateMTY)

In [ ]:
Majortopics2

In [ ]:
topics_condensed2

## use LLM to try to trim total number of topics down

In [ ]:
TopicSynthesizer_template = """
Role: You are in the role of a UX researcher working with other UX researchers evaluting user feedback to find what users like about our system. 
Our system is an AI system that helps user navigate a database of petroleum engineering documentation.
We need to have a good understanding of what users like and dont like about our system so we can improve it.

Instructions:
Another UX researcher has already broken each of the user messages down into a few topics for each user. 
You will be given a list of sets of all of the topics that each user mentioned topics, and your job is to try to reduce all topics down to a few different major topics that reflect different parts of the system we should focus on. 
The set of major topics you generate should be about different parts of the system and not be redundant. You should try to provide only a few topics. 
For each topic you provide, also add an explanation of why you think that is a major theme and what you think we should do to address it.

Topics List: {topicslist}
Analysis: """

In [ ]:
prompt_templateMTY = PromptTemplate(
    input_variables=["topicslist"],
    template=TopicSynthesizer_template
)

In [ ]:
def useLLM_topicSynthesizer(input, template):

    # print(template.format(topicslist = input))
    
    output = ActiveLLM(
    template.format(
        topicslist=input
        )
    )
    return output

In [ ]:
#AUF_Df['MainTopics'].tolist()

In [ ]:
topics_list = AUF_Df['finalTopics'].tolist()

In [ ]:
Majortopics = useLLM_topicSynthesizer( topics_list ,template=prompt_templateMTY)

In [ ]:
Majortopics

# Additional Prompt templates

In [ ]:
UXSentiment_template = """I will give you some text that is feedback from a user testing a new software system. Some of these responses are long and cover mutliple topics.
Analyze the sentiment expressed in the user feedback and for each topic and provide classification of the overall sentiment of all the topics into 1 of 4 categories:
"positive sentiment", "negative sentiment", "neutral sentiment", or "mixed sentiment". 
If the text seems like it was not about the software then say simply say "NA". 
first write your analysis, then classify the text, and then combine the two into a single list like so: [Analysis, Classification].
Below are some examples to help you.

Example 1: 
    User Feedback: "The search function was excellent, but the insight feature was poor"
    Output:["the user like the search function but did not like the input function, and seemd to be equally stongly opinionated on both so this statement is of mixed sentiment" , "mixed sentiment"]

Example 2:
    User Feedback: "I really liked the layout of the user interface. The software was fast and responsive, and the functions did exactly what I expected."
    Output: ["The user expressed positive sentiment about all of the things they mentioned. this statement is postively sentiment" , "positive sentiment"]

Example 3:
    User Feedback: "The system was very difficult to use and not intuitive at all."
    Output: ["The user described the system as ndifficult and not intuitive. This statement is negatively sentiment." , "negative sentiment"]

Example 4:
    User Feedback: "I like ice cream."
    Analysis: ["The statement does not seem to be about the software system." , "NA"]

Example 5:
    User Feedback: "The software was fine and I finished my work."
    Analysis: ["The user did not express any strong sentiments about the system. This statement is of neutral sentiment." , "neutral sentiment"]

User Feedback: {userfeedback}
Analysis: """

In [ ]:
UX_topic_prompt = """
Role: You are in the role of a UX researcher evaluting user feedback to find what users like about our system. Those likes and displikes will be eventually turned into design decisions.
Our system is an AI system that helps user navigate a database of petroleum engineering documentation.

Instructions:
I will give you some text that is feedback from a user testing a new software system. 
This feedback may be long and cover mutliple topics. In that case break down the response into the component topics and return only 1-4 main topics.
Also, summarize those main topics so that each one is only one or two words long, and make sure that none of the main topics are redundant.
Think through this process step by step.
Finally, Once you have determined a set of main topics, return those main topics in a list form like so: ["main topic 1", "main topic 2", "main topic 3"]. 
Do not return the users original feedback or any additional explanation.

Here are two examples:

Example1: 
    User Feedback: "The search function was excellent, but the insight feature was poor"
    Output: ["search function", "insight feature"]

Example 2:
    User Feedback: "The system was very difficult to use and not intuitive at all."
    Output: ["the system"]

User Feedback: {userfeedback}
Output: """

# Scraps

In [ ]:
AUF_Df['MainTopics'] = AUF_Df['Topics'].apply(lambda x: str.split(str.split(x, 'Final Output:')[-1].strip(' []\"').replace('"', '').replace(' ', ''),','))

In [ ]:
str.split(str.split(AUF_Df['Topics'][2], 'Final Output:')[-1].strip(' []\"').replace('"', '').replace(' ', ''),',')

In [ ]:
ast.literal_eval(AUF_Df['Topics'][2])

In [ ]:
prompt_template = PromptTemplate(
    template=UXSentiment_template
)

In [ ]:
AUF_Dft3 = AUF_Df.iloc[1:3,]

In [ ]:
AUF_Dft3['Topics'] = AUF_Dft3['Message'].apply(usemistral, template=prompt_templateT)

In [ ]:
AUF_Dft3['Topics']

In [ ]:
UF_Df['Analysis' ,'Valence'] = UF_Df['Message'].apply(usemistral, template=prompt_template)

In [ ]:
UF_Df

In [ ]:
UF_Df.iloc[2,2]

In [ ]:
UF_Df.iloc[2,4]

In [ ]:
# UF_Df['Analysis']

In [ ]:
AUF_Dft = AUF_Df.iloc[:1,]

In [ ]:
AUF_Dft['Analysis' ,'Valence'] = AUF_Dft['Message'].apply(usemistral, template=prompt_template)

In [ ]:
AUF_Dft.iloc[0,1]

In [ ]:
AUF_Dft.iloc[0,2]

In [ ]:
AUF_Dft2 = AUF_Df.iloc[:2,]

In [ ]:
AUF_Dft2['Topics'] = AUF_Dft['Message'].apply(usemistral, template=prompt_templateT)

In [ ]:
AUF_Dft2['Message'][0]

In [ ]:
AUF_Dft2['Topics'][0]

In [ ]:
AUF_Dft2['Message'][1]

In [ ]:
AUF_Dft2['Topics'][1]

In [ ]:
UF_Df['Valence'][2]

In [ ]:
UF_Df['Message'][0]

In [ ]:
UF_Df['Message'][1]

In [ ]:
UF_Df['Message'][3]

In [ ]:
mistral_bot(
    prompt_template.format(
        userfeedback=
    )
)

In [ ]:
print(llm('Hello, how are you?'))

In [ ]:
teststore = llm("say testing")

In [ ]:
mistral_bot = Ollama(base_url='http://localhost:11434',
             model="mistral")

In [ ]:
print(mistral_bot("say I am mistral"))

In [ ]:
print(mistral_bot({
    "role": "user",
    "content": "why is the sky blue?"
}))

In [ ]:
template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: {previous_context}

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query","previous_context"],
    template=template
)

In [ ]:
print(
    prompt_template.format(
        query="Who was the first president of the united states?",
        previous_context=""
    )
)

In [ ]:
chat_history = ""

In [ ]:
myquery = "Who was the first president of the united states?"
current_response = mistral_bot(
    prompt_template.format(
        query=myquery,
        previous_context = chat_history
        
    )
)
chat_history += "User Question:" + myquery + "\n"
chat_history += "LLM Response:" + current_response + "\n"
print(current_response)

In [ ]:
print(
    prompt_template.format(
        query="Did he write lord of the rings?",
        previous_context=chat_history
    )
)

In [ ]:
myquery = "What dog was in the sandlot movie?"

In [ ]:
print(
    prompt_template.format(
        query="myquery",
        previous_context=chat_history
    )
)

In [ ]:
if len(chat_history) > 3000:
    print("youre are running out of tokens, just a heads up.")
current_response = mistral_bot(
    prompt_template.format(
        query=myquery,
        previous_context = chat_history
        
    )
)
chat_history += myquery
chat_history += current_response
print(current_response)

In [ ]:
# myquery = input()
# if len(chat_history) > 3000:
#     print("youre are running out of tokens, just a heads up.")
# current_response = mistral_bot(
#     prompt_template.format(
#         query=myquery,
#         previous_context = chat_history
        
#     )
# )
# chat_history += myquery
# chat_history += current_response
# print(current_response)

In [ ]:
# OLd User sent template
#UXSentiment_template = """I will give you some text that is feedback from a user testing a new software system. 
# Analyze the emotional valence expressed in the user feedback and then classify the user feedback based on the emotional valence of the text into 1 of 4 categories:
# "positive valence", "negative valence", "neutral valence", or of "mixed valence". 
# If the text seems like it was not about the software then say simply say "NA". 
# first write your analysis, then classify the text, and then combine the two into a single list like so: [Analysis, Classification].
# Below are some examples to help you.

# Example 1: 
#     User Feedback: "The search function was excellent, but the insight feature was poor"
#     Output:["the user like the search function but did not like the input function, and seemd to be equally stongly opinionated on both so this statement is of mixed valence", "mixed valence"]
#     Analysis: "mixed valence"

# Example 2:
#     User Feedback: "I really liked the layout of the user interface. The software was fast and responsive, and the fucntions did exactly what I expected."
#     Analysis: "positive valence"

# Example 3:
#     User Feedback: "The system was very difficult to use and not intuitive at all."
#     Analysis: "negative valence"

# Example 4:
#     User Feedback: "I like cookies."
#     Analysis: "NA"

# Example 5:
#     User Feedback: "The software was fine and I finished my work."
#     Analysis: "neutral valence"

# User Feedback: {userfeedback}
# Analysis: """

In [ ]:
# UserID = []
# System = []
# Message = []

# for i in range(0,len(filelines)):
#     if filelines[i] == '':
#         continue
#     elif filelines[i][-8:] == 'GMT 2023' :
#         UserID.append(int(filelines[i].split(' ')[0].split('-')[1]))
#         System.append(filelines[i].split(' ')[2][0])
#     else:
#         Message.append(filelines[i])
        
                      
        

In [ ]:
#ux sentiment template 2
# UXSentiment_template = """I will give you some text that is feedback from a user testing a new software system. 
# Analyze the emotional valence expressed in the user feedback and then classify the user feedback based on the emotional valence of the text into 1 of 4 categories:
# "positive valence", "negative valence", "neutral valence", or "mixed valence". 
# If the text seems like it was not about the software then say simply say "NA". 
# first write your analysis, then classify the text, and then combine the two into a single list like so: [Analysis, Classification].
# Below are some examples to help you.

# Example 1: 
#     User Feedback: "The search function was excellent, but the insight feature was poor"
#     Output:["the user like the search function but did not like the input function, and seemd to be equally stongly opinionated on both so this statement is of mixed valence" , "mixed valence"]

# Example 2:
#     User Feedback: "I really liked the layout of the user interface. The software was fast and responsive, and the fucntions did exactly what I expected."
#     Output: ["The user expressed positive sentiment about all of the things they mentioned. this statement is postively valenced" , "positive valence"]

# Example 3:
#     User Feedback: "The system was very difficult to use and not intuitive at all."
#     Output: ["The user described the system as ndifficult and not intuitive. This statement is negatively valenced." , "negative valence"]

# Example 4:
#     User Feedback: "I like ice cream."
#     Analysis: ["The statement does not seem to be about the software system." , "NA"]

# Example 5:
#     User Feedback: "The software was fine and I finished my work."
#     Analysis: ["The user did not express any strong sentiments about the system. This statement is of neutral valence." , "neutral valence"]

# User Feedback: {userfeedback}
# Analysis: """

In [ ]:
UF_Df

In [ ]:
UserID

In [ ]:
for i in range(0,15):
    if i % 3 == 0:
        print(i)

In [ ]:
UserID

In [ ]:
filelines[0].split(' ')[0].split('-')[1]

In [ ]:
filelines[0].split(' ')[2][0]

In [ ]:
os.getcwd()
# dir_path = os.path.dirname(os.path.realpath(__file__))


In [ ]:
dir_path = os.path.dirname(os.path.realpath("FAKE_AthenUserFeedback.txt"))

In [ ]:
json.load("Downloads/FAKE_AthenUserFeedback.txt")

In [ ]:
# Opening JSON file
f = open('Downloads/FAKE_AthenUserFeedback.txt')
 
# returns JSON object as 
# a dictionary
data = json.load(f)
 
# Iterating through the json
# list
for i in data['emp_details']:
    print(i)
 
# Closing file
f.close()

In [ ]:

# FormLLM_Template = """

# Instructions: {}

# Context: {}

# User Input: {}

# Output: """

In [ ]:

# TwoKilo_Template = """

# Instructions: {
# You are a form checker, your job is to ...
# Here are the instructions for how to fill out a 2 Kilo form. 
# Here are detail about how the information in the for is used.

# Here are some examples of good an bad responses to certain fields of the 2 kilo form

# Example1

# Example 2

# Example 3
# }

# Context: {Here is information about the role that the current sailor serves,
# Here is infromatoin about the ship and systems that they work on, and 
# Here is infromatino about the jargon or slang associated with that role
# Here is the infromation the sailor has already put into the form}

# User Input: {The field the user is currently filling is ...

# here is the users input to the field
# }

# Output: """